In [171]:
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt

In [174]:
df = pd.read_csv("HistoricalData.csv")
df = df.rename(columns={"Default=1": "Default"})
#drop columns
df = df.drop(columns=["Unnamed: 10", "Unnamed: 11"])
# df = df[:4500]

df.head()

,Firm #,E/A,ROA,D/A,OPM,ATR,Size,QR,CR,Default
0,1,0.55,0.43,0.45,0.27,1.59,807.11,1.98,2.12,0
1,2,0.42,0.61,0.58,0.28,2.17,1040.68,1.31,1.87,0
2,3,0.83,0.52,0.17,0.35,1.51,864.26,1.89,2.16,0
3,4,0.51,0.86,0.49,0.32,2.69,1201.17,2.54,2.97,0
4,5,0.57,0.52,0.43,0.24,2.12,1022.42,1.49,2.47,0


In [125]:
# # calculate the mean of the columns for default and non-defult
# mean_default = df[df["Default"] == 1].mean()
# mean_non_default = df[df["Default"] == 0].mean()
# print(mean_default, mean_non_default)

In [126]:
# predicted_df = pd.read_csv("predictions.csv")
# # predicted_df

# # Apply sigmoid to the predicted values
# predicted_df['predicted_prob'] = 1 / (1 + np.exp(-predicted_df['Predicted']))

# # Now predicted_prob will be between 0 and 1
# print(predicted_df[['Predicted', 'predicted_prob', 'Actual']])

In [175]:
# test the correlation between each column in df
correlation_matrix = df.corr()
print(correlation_matrix)
# it seems that E/A and D/A are highly correlated to each other and we can drop one of them
# also, ATR, Size, and CR are less correlated to Default and we can drop them as well
df = df.drop(columns=["D/A", "ATR", "Size", "CR", "Firm #"])

           Firm #       E/A       ROA       D/A       OPM       ATR      Size  \
Firm #   1.000000 -0.013341  0.019614  0.013341  0.017605  0.000017 -0.001784   
E/A     -0.013341  1.000000 -0.322527 -1.000000 -0.196210 -0.250941 -0.399160   
ROA      0.019614 -0.322527  1.000000  0.322527  0.883347  0.091613  0.517479   
D/A      0.013341 -1.000000  0.322527  1.000000  0.196210  0.250941  0.399160   
OPM      0.017605 -0.196210  0.883347  0.196210  1.000000 -0.358687  0.404990   
ATR      0.000017 -0.250941  0.091613  0.250941 -0.358687  1.000000  0.194580   
Size    -0.001784 -0.399160  0.517479  0.399160  0.404990  0.194580  1.000000   
QR      -0.001867  0.196932 -0.000978 -0.196932 -0.087575  0.194861  0.304658   
CR      -0.020892  0.205772  0.028437 -0.205772 -0.101706  0.289805  0.189370   
Default  0.010855 -0.096968 -0.041513  0.096968 -0.047379  0.021253 -0.010492   

               QR        CR   Default  
Firm #  -0.001867 -0.020892  0.010855  
E/A      0.196932  0.205772 

In [176]:
df.head()

,E/A,ROA,OPM,QR,Default
0,0.55,0.43,0.27,1.98,0
1,0.42,0.61,0.28,1.31,0
2,0.83,0.52,0.35,1.89,0
3,0.51,0.86,0.32,2.54,0
4,0.57,0.52,0.24,1.49,0


In [206]:
# predict model using linear regression 
# x is all columns except Default and y is Default
# using the first 4500 rows of the dataset for training and the rest for testing
x = df[['E/A','ROA','OPM','QR']]
y = df['Default']
model = LogisticRegression()
model.fit(x, y)
# # generate the predicted values and store them in a new column called Predicted
# df['Predicted'] = model.predict(x)
# # apply sigmoid to the predicted values
# df['Predicted'] = 1 / (1 + np.exp(-df['Predicted']))
# df
# # model.coef_
# # model.intercept_

predicted_probs = model.predict_proba(x)[:, 1]  # Probability of Default (class=1)

# Attach to your data
df['predicted_prob'] = predicted_probs

# View some results
print(df[['predicted_prob', 'Default']].head())
# print each coefficient of the model
for feature, coef in zip(x.columns, model.coef_[0]):
    print(f"Feature: {feature}, Coefficient: {coef}")
print("Model Coefficients: ", model.coef_)
print("Model Intercept: ", model.intercept_)


   predicted_prob  Default
0        0.053970        0
1        0.102086        0
2        0.019226        0
3        0.026623        0
4        0.064827        0
Feature: E/A, Coefficient: -3.399888994876017
Feature: ROA, Coefficient: -1.0406879874456771
Feature: OPM, Coefficient: -1.0299288035961345
Feature: QR, Coefficient: -0.6645227895179829
Model Coefficients:  [[-3.39988899 -1.04068799 -1.0299288  -0.66452279]]
Model Intercept:  [1.04741505]


In [205]:
# print all the rows where "Default" is 1
risk = df[df["Default"] == 1]
print(risk)
# what is the minimum value of predicted for risk
# print(risk["Predicted"].min())
# print a summary of the risk
print(risk.describe())

       E/A   ROA   OPM    QR  Default  predicted_prob
11    0.30  0.78  0.40  1.68        1        0.090080
19    0.46  0.87  0.53  1.84        1        0.039524
22    0.53  0.65  0.32  1.96        1        0.044659
31    0.71  0.38  0.18  1.67        1        0.044912
127   0.63  0.64  0.30  1.73        1        0.038451
...    ...   ...   ...   ...      ...             ...
4882  0.71  0.42  0.24  2.02        1        0.032511
4907  0.44  0.69  0.30  1.47        1        0.079260
4936  0.54  0.56  0.23  2.09        1        0.047559
4967  0.60  0.57  0.30  1.99        1        0.038528
4980  0.60  0.94  0.46  2.54        1        0.015791

[211 rows x 6 columns]
              E/A         ROA         OPM          QR  Default  predicted_prob
count  211.000000  211.000000  211.000000  211.000000    211.0      211.000000
mean     0.552085    0.550521    0.276398    1.873460      1.0        0.055538
std      0.106839    0.193758    0.100349    0.385535      0.0        0.026790
min      0.2

In [ ]:
# assign each coefficient to a variable from the model
a = model.intercept_[0]
b1 = model.coef_[0][0]
b2 = model.coef_[0][1]
b3 = model.coef_[0][2]
b4 = model.coef_[0][3]
print(a, b1, b2, b3, b4)

Feature: E/A, Coefficient: -3.399888994876017
Feature: ROA, Coefficient: -1.0406879874456771
Feature: OPM, Coefficient: -1.0299288035961345
Feature: QR, Coefficient: -0.6645227895179829


1.0474150520739245 -3.399888994876017 -1.0406879874456771 -1.0299288035961345 -0.6645227895179829


In [210]:
# load "Competition data.csv"
df2 = pd.read_csv("Competition data.csv")
df2

,Client,Default,E/A,ROA,D/A,OPM,ATR,Size,QR,CR
0,Client 1,0,0.62,0.65,0.38,0.30,2.14,1040.22,2.03,2.51
1,Client 2,0,0.64,0.91,0.36,0.55,1.67,1015.47,1.56,2.45
2,Client 3,0,0.64,0.92,0.36,0.50,1.82,893.21,1.49,2.34
3,Client 4,0,0.16,0.50,0.84,0.20,2.52,1051.25,2.80,2.41
4,Client 5,0,0.61,0.57,0.39,0.28,2.02,990.66,2.06,2.41
...,...,...,...,...,...,...,...,...,...,...
495,Client 496,0,0.31,0.61,0.69,0.29,2.11,917.03,1.56,1.78
496,Client 497,0,0.72,0.75,0.28,0.40,1.85,943.02,2.08,3.13
497,Client 498,0,0.76,0.35,0.24,0.21,1.72,843.28,2.58,2.44
498,Client 499,0,0.54,0.55,0.46,0.35,1.59,927.16,2.26,2.84


In [226]:
# for row in df2.iterrows():
#    EA = df2.loc[row[0], "E/A"]
#    ROA = df2.loc[row[0], "ROA"]
#    OPM = df2.loc[row[0], "OPM"]
#    QR = df2.loc[row[0], "QR"]
#    # calculate the predicted value using the model
#    predicted = a + b1 * EA + b2 * ROA + b3 * OPM + b4 * QR
#    # print(predicted)
#    # add the predicted value to the dataframe
#    df2.loc[row[0], "Predicted"] = predicted
x = df2[['E/A','ROA','OPM','QR']]
y = df2['Default']
predicted_probs = model.predict_proba(x)[:, 1]
df2['predicted_prob'] = predicted_probs

risk2 = df2[df2["Default"] == 1]
risk2[risk2["predicted_prob"] > 0.1].describe()
# risk2.describe()

,Default,E/A,ROA,D/A,OPM,ATR,Size,QR,CR,predicted_prob
count,18.0,18.000000,18.000000,18.000000,18.000000,18.00000,18.000000,18.000000,18.000000,18.000000
mean,1.0,0.223889,0.501111,0.776111,0.238889,2.12500,990.775556,1.792778,2.231111,0.163597
std,0.0,0.094877,0.168589,0.094877,0.082952,0.29857,86.644074,0.339096,0.469767,0.053190
min,1.0,0.070000,0.170000,0.600000,0.100000,1.61000,863.710000,1.280000,1.430000,0.109174
25%,1.0,0.157500,0.420000,0.712500,0.175000,1.94250,929.795000,1.492500,2.057500,0.124172
50%,1.0,0.195000,0.500000,0.805000,0.235000,2.10000,976.965000,1.780000,2.200000,0.142985
75%,1.0,0.287500,0.657500,0.842500,0.300000,2.28000,1046.150000,1.985000,2.595000,0.191028
max,1.0,0.400000,0.810000,0.930000,0.410000,2.63000,1149.530000,2.420000,3.060000,0.310092
